In [157]:
import numpy as np
import pandas as pd
from math import sqrt, log
from itertools import chain, product
from collections import defaultdict

df1 = pd.read_excel('corpus.xlsx', sheetname='shh',index_col=0)
df2 = pd.read_excel('corpus.xlsx', sheetname='loo',index_col=0)

print(df1.head())
print(df2.head())

                                                        DESCRIPTION
UNIQUEID                                                           
BAR006VBA-ADAA                 MIST ELIMINATOR A VAPOR EXTRACTION A
BAR006VBB-AAAC                             LUBE OIL #1 PUMP PUMPS B
BAR007WGB-AD                                              HEADERS B
BAR007WHB-AAAD    DUCT BURNER SCANNER COOLING FAN #1 HRSG B DUCT...
BAR007MAB-AAAAAJ       LUBE OIL SKID B1 BOILER FEED PUMP B1 PUMPS B
                                                            DESCRIPTION
UNIQUEID                                                               
BAR006-ASA-HV-21002A  VALVE zero ISOLATION SUCTION STRAINER DIFFEREN...
BAR006-ASA-MCMA-1AB   U3 one - B HRSG FEEDWATER PUMP SUCTION LINE RO...
BAR006-ASB-HV-22002B  U3 two - A HRSG #1 FEEDWATER PUMP SUCTION STRA...
BAR006-ASB-MCMB-10B   U3 three - A HRSG #2 FEEDWATER PUMP SUCTION ST...
kfgjdfkg              U3 four - A HRSG #2 FEEDWATER PUMP SUCTION STR...


In [326]:
def cosine_sim(u,v):
    return np.dot(u,v) / (sqrt(np.dot(u,u)) * sqrt(np.dot(v,v)))

def ngrams(sentence, n):
    return zip(*[sentence.split()[i:] for i in range(n)])

def corpus2vectors(corpus):
    def vectorize(sentence, vocab):
        return [sentence.split().count(i.lower()) for i in vocab]
    vectorized_corpus = []
    vocab = sorted(set(chain(*[i.lower().split() for i in corpus])))
    for i in corpus:
        vectorized_corpus.append((i, vectorize(i, vocab)))
    return vectorized_corpus, vocab

def create_test_corpus(input_column):
    all_sents = list(input_column)
    corpus, vocab = corpus2vectors(all_sents)
    return corpus, vocab      
        
def test_cosine():
    score,highestScore = 0,0
    counter = 0
    bestMatch = []
    allMatches = pd.DataFrame(columns=['UID','index','indexOfMatch','score'])
    matches = np.array([0,0,0.])
    
    for sentx, senty in product(corpus, corpus):
        print("______________________________________")
        print("counter:",counter)
        print("=X=",sentx)
        print("=Y=",senty)
        
        index1 = counter // len(corpus)
        index2 = counter % len(corpus)
        
        if (index2 == 0):
            highestScore = 0

        score  = cosine_sim(sentx[1], senty[1])
        print("WORDS SENTENCE 1: ",sentx[0])
        print("WORDS SENTENCE 2: ",senty[0])
        #print("NUMS SENTENCE 1: ",sum(sentx[1]))
        #print("NUMS SENTENCE 2: ",sum(senty[1]))
        print("cosine = ", score)
        if (score >= 2):
            if (score < 2):
                highestScore = score
                print('index1:',index1)
                print('uid is: ',)
                uid = df2.index.values[index1]
                bestMatch = np.array(uid, index1, index2, highestScore)
                print("-- New High Score:", bestMatch) 
        
        uid = df2.index.values[index1]
        bestMatch = np.array([index1, index2, score])
        matches = np.vstack((matches, bestMatch))
        
        counter = counter + 1
    
    return matches


sen1 = "this is a test sentence all the way down are turtles haha ok bye"
sen2 = "this is another test sentence ok by turtles down here"
sen3 = "HRSG POWER AUGMENTATION STEAM"
sen4 = "U6-B HRSG POWER AUGMENTATION VENT VALVE 480V BREAKER (ASB-MV-125)"



corpus,vocab = create_test_corpus(df2['DESCRIPTION'].str.lower())
test_cosine()

______________________________________
counter: 0
=X= ('u6-a hrsg power augmentation steam manual supply isolation valve', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])
=Y= ('u6-a hrsg power augmentation steam manual supply isolation valve', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])
WORDS SENTENCE 1:  u6-a hrsg power augmentation steam manual supply isolation valve
WORDS SENTENCE 2:  u6-a hrsg power augmentation steam manual supply isolation valve
cosine =  1.0
______________________________________
counter: 1
=X= ('u6-a hrsg power augmentation steam manual supply isolation valve', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])
=Y= ('u6-a hrsg power augmentation steam blocking valve 480v breaker (asa-mv-100)', [1, 0, 0, 0, 

WORDS SENTENCE 1:  u6-b power augmentation steam blocking valve 480v breaker (asb-mv-100)
WORDS SENTENCE 2:  u6-a1 combustion turbine exhaust frame blower
cosine =  0.0
______________________________________
counter: 102
=X= ('u6-b power augmentation steam blocking valve 480v breaker (asb-mv-100)', [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])
=Y= ('u6-b1 combustion turbine exhaust frame blower', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-b power augmentation steam blocking valve 480v breaker (asb-mv-100)
WORDS SENTENCE 2:  u6-b1 combustion turbine exhaust frame blower
cosine =  0.0
______________________________________
counter: 103
=X= ('u6-b power augmentation steam blocking valve 480v breaker (asb-mv-100)', [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

cosine =  0.0
______________________________________
counter: 252
=X= ('u6-b hrsg power augmentation vent valve 480v breaker (asb-mv-125)', [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0])
=Y= ('u6-b1 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-b hrsg power augmentation vent valve 480v breaker (asb-mv-125)
WORDS SENTENCE 2:  u6-b1 flame scanner cooling fan
cosine =  0.0
______________________________________
counter: 253
=X= ('u6-b hrsg power augmentation vent valve 480v breaker (asb-mv-125)', [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0])
=Y= ('u6-b2 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

counter: 410
=X= ('u6-a1 combustion turbine exhaust frame blower', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0])
=Y= ('u6-a hrsg power augmentation pressure control air operated valve (asa-tv-101)', [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])
WORDS SENTENCE 1:  u6-a1 combustion turbine exhaust frame blower
WORDS SENTENCE 2:  u6-a hrsg power augmentation pressure control air operated valve (asa-tv-101)
cosine =  0.0
______________________________________
counter: 411
=X= ('u6-a1 combustion turbine exhaust frame blower', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0])
=Y= ('u6-a hrsg power augmentation', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

=X= ('u6-a2 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
=Y= ('u6-a combustion turbine turbine dewpoint sensor 120v breaker', [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-a2 flame scanner cooling fan
WORDS SENTENCE 2:  u6-a combustion turbine turbine dewpoint sensor 120v breaker
cosine =  0.0
______________________________________
counter: 564
=X= ('u6-a2 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
=Y= ('u6-b combustion turbine turbine dewpoint sensor 120v breaker', [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-a2 flame scanner cooling fa

=Y= ('u6-a2 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-a2 combustion turbine bearing area cooling fan
WORDS SENTENCE 2:  u6-a2 flame scanner cooling fan
cosine =  0.5070925528371099
______________________________________
counter: 716
=X= ('u6-a2 combustion turbine bearing area cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0])
=Y= ('u6-b1 flame scanner cooling fan', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
WORDS SENTENCE 1:  u6-a2 combustion turbine bearing area cooling fan
WORDS SENTENCE 2:  u6-b1 flame scanner cooling fan
cosine =  0.33806170189140655
______________________________________
counter: 717
=X= ('u6-a2 combustion turbine bearing area cooling fan',

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.63245553],
       ...,
       [28.        , 26.        ,  0.88888889],
       [28.        , 27.        ,  0.88888889],
       [28.        , 28.        ,  1.        ]])

In [358]:
# import numpy as np
import pandas as pd
from math import sqrt, log
from itertools import chain, product
from collections import defaultdict

# Load the main 2 tables I will be comparing
df1 = pd.read_excel('corpus.xlsx', sheetname='shh',index_col=0)
df2 = pd.read_excel('corpus.xlsx', sheetname='loo',index_col=0)

# method of calculating similarity of the one-hot encoding
def cosine_sim(u,v):
    return np.dot(u,v) / (sqrt(np.dot(u,u)) * sqrt(np.dot(v,v)))

# Vectorize to one-hot binary encoding as numbers work better than strings!
def corpus2vectors(corpus):
    def vectorize(sentence, vocab):
        return [sentence.split().count(i.lower()) for i in vocab]
    vectorized_corpus = []
    # Remobe dupes and sort
    vocab = sorted(set(chain(*[i.lower().split() for i in corpus])))
    for i in corpus:
        vectorized_corpus.append((i, vectorize(i, vocab)))
    return vectorized_corpus, vocab

# Basically just combine these two datasets to get a single corpus
def create_test_corpus(data1,data2):
    all_sents = list(data1) + list(data2)
    corpus, vocab = corpus2vectors(all_sents)
    return corpus, vocab      

# Workhouse function, cycle through all combinations and compute scores
def test_cosine(data1,data2):
    score,highestScore = 0,0
    counter = 0
    newMatch = []
    #allMatches = pd.DataFrame(columns=['UID','index','indexOfMatch','score'])
    matches = np.array([0,0,0.])
    
    # Cycle through each row in data1
    for dat1_ix, dat1 in enumerate(data1):
        print("___________________________________________________")
        print("dat1:",dat1)

        # For each iteration of data1, cycle through all of data2 below
        for dat2_ix, dat2 in enumerate(data2):
            print("_______________________")

            print("Phrase 1:",dat1)
            print("Phrase 2:",dat2)
            print('length df1:',len(df1),'   length df2:',len(df2))
            print("counter:",counter)
            
            # Calculate cosime similarity from each line to the other
            vec1 = corpus[dat1_ix][1]
            print("vec2:",(len(dat1) - 1) + (dat2_ix - 2))
            vec2 = corpus[(len(dat1) - 1) + (dat2_ix - 2)][1]
            score  = cosine_sim(vec1, vec2)
            print("cosine = ", score)
            
            #uid = df2.index.values[index1]
            newMatch = np.array([dat1_ix, dat2_ix, score])
            matches = np.vstack((matches, newMatch))
        
        counter = counter + 1
    
    return matches

# Lower
data1 = df1['DESCRIPTION'].str.lower()
data2 = df2['DESCRIPTION'].str.lower()                   
corpus,vocab = create_test_corpus(data1,data2)

test_cosine(data1,data2)

___________________________________________________
dat1: mist eliminator a vapor extraction a
_______________________
Phrase 1: mist eliminator a vapor extraction a
Phrase 2: u6-a hrsg power augmentation steam manual supply isolation valve
length df1: 25    length df2: 29
counter: 0
vec2: 33
cosine =  0.0
_______________________
Phrase 1: mist eliminator a vapor extraction a
Phrase 2: u6-a hrsg power augmentation steam blocking valve 480v breaker (asa-mv-100)
length df1: 25    length df2: 29
counter: 0
vec2: 34
cosine =  0.0
_______________________
Phrase 1: mist eliminator a vapor extraction a
Phrase 2: u6-a hrsg power augmentation vent valve 480v breaker (asa-mv-125)
length df1: 25    length df2: 29
counter: 0
vec2: 35
cosine =  0.0
_______________________
Phrase 1: mist eliminator a vapor extraction a
Phrase 2: u6-b power augmentation steam blocking valve 480v breaker (asb-mv-100)
length df1: 25    length df2: 29
counter: 0
vec2: 36
cosine =  0.0
_______________________
Phrase 1: m

IndexError: list index out of range

In [362]:
len(data2) + len(data1)

54